In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
from data_objects.article import Article
from data_objects.legislation import Legislation
from data_objects.public_consultation import PublicConsultation
from data_objects.base import Base

import requests

from sqlalchemy import create_engine, select, union_all, func, delete
from sqlalchemy.orm import Session, aliased

from bs4 import BeautifulSoup
import time
from configparser import ConfigParser

config = ConfigParser()
config.read("config.ini")

['config.ini']

In [2]:
engine = create_engine(config.get('DEFAULT', 'db_file'))

### Delete Duplicate records based on 'scrap_url'

In [3]:
with Session(engine) as sess:
    stmt = select(func.min(Legislation.id)).group_by(Legislation.scrap_url).having(func.count(Legislation.scrap_url)>1)
    rows = sess.execute(stmt)
    for row in rows:
        print(f"{row[0]}")

In [4]:
with Session(engine) as sess:
    subq = select(func.min(Legislation.id).label("id")).group_by(Legislation.scrap_url).having(func.count(Legislation.scrap_url)>1).subquery()
    stmt = select(Legislation).join(subq, subq.c.id==Legislation.id)

    rows = sess.execute(stmt).scalars()
    print("rows to be deleted")
    for row in rows:
        print(f"{row}")
    delete_stmt = delete(Legislation).where(Legislation.id.in_(
        select(func.min(Legislation.id).label("id")).group_by(Legislation.scrap_url).having(func.count(Legislation.scrap_url)>1)
    ))
    sess.execute(delete_stmt)

    rows = sess.execute(stmt).scalars()
    # verify that rows are actually deleted
    for row in rows:
        print(f"{row}")
    
    sess.commit()


rows to be deleted


In [5]:
#delete associated articles and Public Consultations
with Session(engine) as sess:
    stmt = select(Article).where(Article.legislation_id.in_(
        [704, 701, 700, 697, 696, 702, 698, 703, 705, 699]
    ))
    articles = sess.execute(stmt).scalars()
    for article in articles:
        print(f"{article}")
        for publicConsultation in article.public_consultations:
            sess.delete(publicConsultation)
        sess.delete(article)    
        
    sess.commit()


### Update Title

In [28]:
url = "http://www.opengov.gr/yptp/?p=2822"
def get_correct_title(url:str) -> str:
    r=requests.get(url)
    soup = BeautifulSoup(r.content, features="html.parser")
    titles = soup.select('h3')
    title = titles[len(titles)-1].text
    return title

t = get_correct_title(url)
print(t)

Δημόσιες υπαίθριες συναθροίσεις


In [29]:
def get_correct_title(url:str) -> str:
    r=requests.get(url)
    soup = BeautifulSoup(r.content, features="html.parser")
    titles = soup.select('h3')
    title = titles[len(titles)-1].text
    return title

with Session(engine) as sess:
    subq = select(Legislation.title).group_by(Legislation.title).having(func.count(Legislation.title)>1)
    stmt = select(Legislation).where(Legislation.title.in_(subq))
    legislations_with_duplicate_titles = sess.execute(stmt).scalars()

    for legObj in legislations_with_duplicate_titles:
        print(f"{legObj}")
        correct_title = get_correct_title(legObj.scrap_url)
        legObj.title = correct_title
        print(legObj)
        # break
    
    sess.commit()


Legislation(id=27, title=Ολοκλήρωση δημόσιας ηλεκτρονικής διαβούλευσης, ministry=Υπουργείο Υγείας, date_posted=2022-10-20)
Legislation(id=27, title=Διαβούλευση επί του σχεδίου νόμου: «Δευτεροβάθμια περίθαλψη, ιατρική εκπαίδευση και λοιπές διατάξεις αρμοδιότητας του Υπουργείου Υγείας», ministry=Υπουργείο Υγείας, date_posted=2022-10-20)
Legislation(id=35, title=Ολοκλήρωση δημόσιας ηλεκτρονικής διαβούλευσης, ministry=Εργασίας και Κοινωνικής Ασφάλισης, date_posted=2022-07-13)
Legislation(id=35, title=Δημόσια ηλεκτρονική διαβούλευση για την «ΕΘΝΙΚΗ ΣΤΡΑΤΗΓΙΚΗ ΓΙΑ ΤΙΣ ΕΝΕΡΓΗΤΙΚΕΣ ΠΟΛΙΤΙΚΕΣ ΑΠΑΣΧΟΛΗΣΗΣ», ministry=Εργασίας και Κοινωνικής Ασφάλισης, date_posted=2022-07-13)
Legislation(id=40, title=Ολοκλήρωση δημόσιας ηλεκτρονικής διαβούλευσης, ministry=Εργασίας και Κοινωνικής Ασφάλισης, date_posted=2022-06-20)
Legislation(id=40, title=Δημόσια ηλεκτρονική διαβούλευση επί του σχεδίου Κοινής Απόφασης των Υπουργών Εργασίας και Κοινωνικών Υποθέσεων και Υγείας με τίτλο «Εφαρμογή των διατάξεων της περ

remove new line from title

In [18]:
with Session(engine) as sess:
    stmt = select(Legislation)#.where(Legislation.id==1035)
    legislations = sess.execute(stmt).scalars()

    for legObj in legislations:
        legObj:Legislation
        # print(legObj)
        # print(legObj.title)
        # print()
        legObj.title = legObj.title.replace("\r","").replace("\n","").replace("\r\n","").strip()
        # print(legObj.title)
        sess.commit()


In [9]:
txt = """Ενσωμάτωση στην εθνική έννομη τάξη της Οδηγίας (ΕΕ) 2019/884 του Ευρωπαϊκού Κοινοβουλίου και 
του Συμβουλίου της 17ης Απρίλιου 2019 για την τροποποίηση της απόφασης - πλαισίου 2009/315/
ΔΕΥ του Συμβουλίου όσον αφορά την ανταλλαγή πληροφοριών σχετικά με υπηκόους τρίτων χωρών 
και όσον αφορά το Ευρωπαϊκό Σύστημα Πληροφοριών Ποινικού Μητρώου (ECRIS), και για την 
αντικατάσταση της απόφασης 2009/316/ΔΕΥ του Συμβουλίου (L 151) και λοιπές ρυθμίσεις του 
Υπουργείου Δικαιοσύνης"""
print(txt)
print()
print(txt.replace("\n",""))

Ενσωμάτωση στην εθνική έννομη τάξη της Οδηγίας (ΕΕ) 2019/884 του Ευρωπαϊκού Κοινοβουλίου και 
του Συμβουλίου της 17ης Απρίλιου 2019 για την τροποποίηση της απόφασης - πλαισίου 2009/315/
ΔΕΥ του Συμβουλίου όσον αφορά την ανταλλαγή πληροφοριών σχετικά με υπηκόους τρίτων χωρών 
και όσον αφορά το Ευρωπαϊκό Σύστημα Πληροφοριών Ποινικού Μητρώου (ECRIS), και για την 
αντικατάσταση της απόφασης 2009/316/ΔΕΥ του Συμβουλίου (L 151) και λοιπές ρυθμίσεις του 
Υπουργείου Δικαιοσύνης

Ενσωμάτωση στην εθνική έννομη τάξη της Οδηγίας (ΕΕ) 2019/884 του Ευρωπαϊκού Κοινοβουλίου και του Συμβουλίου της 17ης Απρίλιου 2019 για την τροποποίηση της απόφασης - πλαισίου 2009/315/ΔΕΥ του Συμβουλίου όσον αφορά την ανταλλαγή πληροφοριών σχετικά με υπηκόους τρίτων χωρών και όσον αφορά το Ευρωπαϊκό Σύστημα Πληροφοριών Ποινικού Μητρώου (ECRIS), και για την αντικατάσταση της απόφασης 2009/316/ΔΕΥ του Συμβουλίου (L 151) και λοιπές ρυθμίσεις του Υπουργείου Δικαιοσύνης
